In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xlrd
from scipy.stats import ttest_ind, f_oneway, percentileofscore, shapiro
import math
from datetime import datetime, timedelta
import constants
import functions
import modelling_functions
from numpy.random import randn
import matplotlib.ticker as mtick

import importlib
importlib.reload(constants)
importlib.reload(functions)
importlib.reload(modelling_functions)

import sys
sys.path.insert(0,'..')
import analysis_utils

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

bls_codes= pd.read_csv('../../data/zhaw_lca/bls_codes_14_07.csv').set_index('BLS_Code')

# 1. Load Data

In [2]:
user_dataset, user_purchases_dataset, purchases_dataset, user_purchase_summary_dataset = analysis_utils.get_datasets(constants.dataset_filename_prefix2)

print('Number of households: ', user_purchase_summary_dataset.r2n_user.nunique())

/opt/homebrew/Caskroom/miniforge/base/envs/cenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (38,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Number of households:  462


# 2. Exclusion Criteria
First, we remove households that don't meet the criteria:
* On average at least 1 shopping day every 2 weeks
* At least 1 year of shopping history

In [3]:
exclusion_conditions = (
    (user_purchase_summary_dataset.shopping_frequency_per_week >= 0.5) 
    & (user_purchase_summary_dataset.receipt_history_duration_days >= 365)    
)

working_user_dataset, working_user_purchases_dataset = analysis_utils.implement_exclusion_criteria(user_purchase_summary_dataset,purchases_dataset,exclusion_conditions)
working_user_dataset = working_user_dataset.sort_values(by='user_total_gwp100_normalised_1000kcal', ascending=False)

print('Number of households: ', working_user_purchases_dataset.r2n_user.nunique())

Number of households:  347


In [4]:
def update_emissions(minor, quantity_in_kg_or_litres, emissions, actual_kcal):

    substitution_percentage = 1
    weight_to_substitute = quantity_in_kg_or_litres*substitution_percentage

    if minor == 'Milk':
        # substitute
        # V-Love Bio · Oat drink · Barista. 610 kcal per litre or kg.
        # 'Plant-based drinks'
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ98', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Milk and yoghurt drinks':
        # substitute
        # V-Love · Vegurt · Coconut-Nature. 1050 kcal per kg
        # 'Plant-based drinks'
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ119', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Yogurt and sour milk':
        # substitute
        # V-Love · Vegurt · Coconut-Nature. 1050 kcal per kg
        # 'Plant-based drinks'
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ119', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Cheese':
        # substitute
        # V-Love The Classic · Plant-based alternative to cheese · with lupines. 2580 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ117', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Soft cheese' :
        # substitute
        # V-Love The Softy · Vegetable alternative to Camembert · Plain. 3520 kcal/kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ115', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Cheese products' :
        # substitute
        # V-Love The Classic · Plant-based alternative to cheese · with lupines. 2580 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ117', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        


    elif minor == 'Cream cheese and cottage cheese':
        # substitute
        # V-Love The Cottage Cubes · Vegetable alternative to cottage cheese · with chives. 1240 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ114', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Butter and margarine':
        # substitute
        # margarine. 7200 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['Q400000', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Cream':
        # substitute
        # V-Love · Plant-based Cuisine. 2750 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ113', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Sausage':
        # substitute
        # V-Love Plant-Based Würstchen. 2220 kcal / kg
        # 'Vegetarian meat alternatives'
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ123', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Meat sliced' :
        # substitute
        # V-Love · Vegetable alternative to cold cuts spicy · Vegan. 2290 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ123', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Beef' :
        # substitute
        # V-Love · Vegetable minced meat · Vegan. 1760 kcal / kg
        # 'Vegetarian meat alternatives'
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ123', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Poultry' :
        # substitute
        # V-Love · Nuggets · Vegan. 2360 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['WW2', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Veal':
        # substitute
        # V-Love · Vegetable minced meat · Vegan. 1760 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ123', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor == 'Pork' :

        # substitute
        # V-Love · Delicious pieces · Vegan. 1690 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ122', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        

    elif minor in ['Fish products','Marine fish','Freshwater fish', 'Seafood, crustaceans and shellfish']:

        # substitute
        # V-Love · Sticks · Vegan. 2300 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ124', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute
        


    elif minor in ['Other types of meat', 'Lamb & sheep', 'Venison']:
        # substitute
        # V-Love · Sticks · Vegan. 2300 kcal / kg
        return emissions*(1-substitution_percentage) + bls_codes.loc['ZZ124', 'GWP100 (kg CO2-eq./kg)'] * weight_to_substitute        

    else:
        return emissions

In [5]:
df = working_user_purchases_dataset.copy()

df['emissions'] = df['Total GWP100']
df['new_emissions'] = df['emissions']
df['new_emissions'] = df.apply(lambda row: update_emissions(row.minor, row.quantity_in_kg_or_litres, row.emissions, row.actual_kcal), axis=1)

# Swap simulations

In [6]:
df_household = df.groupby(by='r2n_user', dropna=False).sum()

df_household['emissions_reduction_everything'] = 1 - df_household['new_emissions']/df_household['emissions']

df_household = df_household.rename(columns={'emissions': 'previous_emissions',                                            
                                            'new_emissions':'new_emissions_everything'})

df_household = df_household[['quantity_in_kg_or_litres', 'previous_emissions', 'new_emissions_everything', 'emissions_reduction_everything',]]

In [7]:
def analyse_substitution(grouping):
    

    household_ids = []
    
    previous_emissions_list = []
    previous_kcals_list = []
    
    emissions_reduction_1grouping = []
    emissions_reduction_2grouping = []
    emissions_reduction_3grouping = []

    emissions_reduction_per_kcal_1grouping = []
    emissions_reduction_per_kcal_2grouping = []
    emissions_reduction_per_kcal_3grouping = []
    
    grouping1_highest_emissions_list = []
    grouping2_highest_emissions_list = []
    grouping3_highest_emissions_list = []
    
    grouping1_highest_emissions_emissions_percentage_list = []
    grouping2_highest_emissions_emissions_percentage_list = []
    grouping3_highest_emissions_emissions_percentage_list = []
    
    grouping1_highest_emissions_kcals_percentage_list = []
    grouping2_highest_emissions_kcals_percentage_list = []
    grouping3_highest_emissions_kcals_percentage_list = []

    grouping1_highest_substituted_list = []
    grouping2_highest_substituted_list = []
    grouping3_highest_substituted_list = []
    
    grouping1_highest_substituted_emissions_percentage_list = []
    grouping2_highest_substituted_emissions_percentage_list = []
    grouping3_highest_substituted_emissions_percentage_list = []
    
    grouping1_highest_substituted_kcals_percentage_list = []
    grouping2_highest_substituted_kcals_percentage_list = []
    grouping3_highest_substituted_kcals_percentage_list = []


    new_emissions_1grouping = []
    new_emissions_2grouping = []
    new_emissions_3grouping = []
    new_emissions_4grouping = []
    new_emissions_5grouping = []

    df_household_grouping = df.groupby(by=['r2n_user', grouping], dropna=False).sum()
    df_household_grouping['emissions_reduction'] = 1 - df_household_grouping['new_emissions']/df_household_grouping['emissions']
    df_household_grouping['emissions_diff'] = df_household_grouping['emissions'] - df_household_grouping['new_emissions']
    df_household_grouping = df_household_grouping.sort_values(by='emissions_diff', ascending=False)
    
    df_household_grouping_diff_sorted = df_household_grouping.sort_values(by='emissions_diff', ascending=False)
    df_household_grouping_emissions_sorted = df_household_grouping.sort_values(by='emissions', ascending=False)
    
    for household in df_household.index:
        household_ids.append(household)
        previous_emissions = df_household_grouping_emissions_sorted.loc[household].emissions.sum()

        previous_emissions_list.append(previous_emissions)

        grouping1_highest_emissions_list.append(df_household_grouping_emissions_sorted.loc[household].index[0])
        grouping1_highest_substituted_list.append(df_household_grouping_diff_sorted.loc[household].index[0])
        grouping1_highest_emissions_emissions_percentage_list.append(df_household_grouping_emissions_sorted.loc[household, 'emissions'][0] / df_household_grouping_emissions_sorted.loc[household, 'emissions'].sum())
        grouping1_highest_substituted_emissions_percentage_list.append(df_household_grouping_diff_sorted.loc[household, 'emissions'][0] / df_household_grouping_diff_sorted.loc[household, 'emissions'].sum())
        
        grouping2_highest_emissions_list.append(df_household_grouping_emissions_sorted.loc[household].index[1])
        grouping2_highest_substituted_list.append(df_household_grouping_diff_sorted.loc[household].index[1])
        grouping2_highest_emissions_emissions_percentage_list.append(df_household_grouping_emissions_sorted.loc[household, 'emissions'][0:2].sum() / df_household_grouping_emissions_sorted.loc[household, 'emissions'].sum())
        grouping2_highest_substituted_emissions_percentage_list.append(df_household_grouping_diff_sorted.loc[household, 'emissions'][0:2].sum() / df_household_grouping_diff_sorted.loc[household, 'emissions'].sum())
        
        grouping3_highest_emissions_list.append(df_household_grouping_emissions_sorted.loc[household].index[2])
        grouping3_highest_substituted_list.append(df_household_grouping_diff_sorted.loc[household].index[2])
        grouping3_highest_emissions_emissions_percentage_list.append(df_household_grouping_emissions_sorted.loc[household, 'emissions'][0:3].sum() / df_household_grouping_emissions_sorted.loc[household, 'emissions'].sum())
        grouping3_highest_substituted_emissions_percentage_list.append(df_household_grouping_diff_sorted.loc[household, 'emissions'][0:3].sum() / df_household_grouping_diff_sorted.loc[household, 'emissions'].sum())       
        
        emissions_reduction_1grouping.append(df_household_grouping_diff_sorted.loc[household].emissions_diff[0].sum() / previous_emissions)
        new_emissions_1grouping.append(previous_emissions - df_household_grouping_diff_sorted.loc[household].emissions_diff[0].sum())

        emissions_reduction_2grouping.append(df_household_grouping_diff_sorted.loc[household].emissions_diff[0:2].sum() / previous_emissions)
        new_emissions_2grouping.append(previous_emissions - df_household_grouping_diff_sorted.loc[household].emissions_diff[0:2].sum())

        emissions_reduction_3grouping.append(df_household_grouping_diff_sorted.loc[household].emissions_diff[0:3].sum() / previous_emissions)
        new_emissions_3grouping.append(previous_emissions - df_household_grouping_diff_sorted.loc[household].emissions_diff[0:3].sum())

     
    df_substitute_groupings = pd.DataFrame(index=household_ids, data={

    'highest_emissions_1'+grouping: grouping1_highest_emissions_list,
    'highest_potential_1'+grouping: grouping1_highest_substituted_list,
    'highest_emissions_emissions_percentage_1'+grouping: grouping1_highest_emissions_emissions_percentage_list,
    'highest_potential_emissions_percentage_1'+grouping: grouping1_highest_substituted_emissions_percentage_list,
        
    'highest_emissions_2'+grouping: grouping2_highest_emissions_list,
    'highest_potential_2'+grouping: grouping2_highest_substituted_list,
    'highest_emissions_emissions_percentage_2'+grouping: grouping2_highest_emissions_emissions_percentage_list,
    'highest_potential_emissions_percentage_2'+grouping: grouping2_highest_substituted_emissions_percentage_list,
        
    'highest_emissions_3'+grouping: grouping3_highest_emissions_list,
    'highest_potential_3'+grouping: grouping3_highest_substituted_list,
    'highest_emissions_emissions_percentage_3'+grouping: grouping3_highest_emissions_emissions_percentage_list,
    'highest_potential_emissions_percentage_3'+grouping: grouping3_highest_substituted_emissions_percentage_list,

        
    'new_emissions_1'+grouping: new_emissions_1grouping,
    'emissions_reduction_1'+grouping: emissions_reduction_1grouping,

    'new_emissions_2'+grouping: new_emissions_2grouping,
    'emissions_reduction_2'+grouping: emissions_reduction_2grouping,

    'new_emissions_3'+grouping: new_emissions_3grouping,
    'emissions_reduction_3'+grouping: emissions_reduction_3grouping,

    })
    
    return df_substitute_groupings

In [8]:
# (i.e. Generic Swap Recommedations - All meat and dairy)
substitute_everything = df_household.copy()

# (i.e. Personalised Swap Recommedations - highest emission food categories)
substitute_r2i = analyse_substitution('r2i_category')

# Reduce Specific To A Basket Product (i.e. real-time swap recommendations)

In [9]:
df_basket_product = df.groupby(by=['receipt_datetime', 'r2n_user', 'product_name_de'], dropna=False).sum()
df_basket_product['emissions_diff'] = df_basket_product['emissions'] - df_basket_product['new_emissions']
df_basket_product = df_basket_product.sort_values(by=['receipt_datetime', 'r2n_user', 'emissions_diff'], ascending=False).reset_index(level='product_name_de')

dates = []
users = []
emissions_diff_top_1products = []
emissions_diff_top_2products = []
emissions_diff_top_3products = []



basket_emissions = []
basket_emissions_reduction = []

for basket in df_basket_product.index.unique():
    date, user = basket
    product_1_emissions_diff = 0
    product_2_emissions_diff = 0
    product_3_emissions_diff = 0
    
    try:
        emissions = df_basket_product.loc[basket].emissions.sum()
        product_1_emissions_diff = df_basket_product.loc[basket].emissions_diff[0]
        product_2_emissions_diff = df_basket_product.loc[basket].emissions_diff[1]
        product_3_emissions_diff = df_basket_product.loc[basket].emissions_diff[2]
       
    except:
        None
        
    basket_emissions.append(emissions)
    basket_emissions_reduction.append(product_1_emissions_diff/emissions)
    dates.append(date)
    users.append(user)
    emissions_diff_top_1products.append(product_1_emissions_diff)
    emissions_diff_top_2products.append(product_1_emissions_diff+product_2_emissions_diff)
    emissions_diff_top_3products.append(product_1_emissions_diff+product_2_emissions_diff+product_3_emissions_diff) 
    
basket_substitutions = pd.DataFrame(data={
    'receipt_datetime':dates, 
    'r2n_user': users, 
    'basket_emissions_reduction': basket_emissions_reduction, 
    'basket_emissions': basket_emissions, 
    'emissions_diff_top1basketproduct':emissions_diff_top_1products, 'emissions_diff_top2basketproduct':emissions_diff_top_2products, 'emissions_diff_top3basketproduct':emissions_diff_top_3products
})

df_household_basket = df_household[['previous_emissions']].join(basket_substitutions.groupby(by='r2n_user').sum()[
    ['emissions_diff_top1basketproduct', 'emissions_diff_top2basketproduct', 'emissions_diff_top3basketproduct', 
    ]])

df_household_basket['new_emissions_top1basketproduct'] = df_household_basket.previous_emissions - df_household_basket.emissions_diff_top1basketproduct
df_household_basket['new_emissions_top2basketproduct'] = df_household_basket.previous_emissions - df_household_basket.emissions_diff_top2basketproduct
df_household_basket['new_emissions_top3basketproduct'] = df_household_basket.previous_emissions - df_household_basket.emissions_diff_top3basketproduct

df_household_basket['emissions_reduction_top1basketproduct'] = 1 - df_household_basket['new_emissions_top1basketproduct']/df_household_basket['previous_emissions']
df_household_basket['emissions_reduction_top2basketproduct'] = 1 - df_household_basket['new_emissions_top2basketproduct']/df_household_basket['previous_emissions']
df_household_basket['emissions_reduction_top3basketproduct'] = 1 - df_household_basket['new_emissions_top3basketproduct']/df_household_basket['previous_emissions']

df_household_basket = df_household_basket[
    [
        'new_emissions_top1basketproduct', 'emissions_reduction_top1basketproduct',
        'new_emissions_top2basketproduct', 'emissions_reduction_top2basketproduct',
        'new_emissions_top3basketproduct', 'emissions_reduction_top3basketproduct',
    ]]


<ipython-input-9-9096e654f39a>:23: PerformanceWarning: indexing past lexsort depth may impact performance.
  emissions = df_basket_product.loc[basket].emissions.sum()
<ipython-input-9-9096e654f39a>:24: PerformanceWarning: indexing past lexsort depth may impact performance.
  product_1_emissions_diff = df_basket_product.loc[basket].emissions_diff[0]
<ipython-input-9-9096e654f39a>:25: PerformanceWarning: indexing past lexsort depth may impact performance.
  product_2_emissions_diff = df_basket_product.loc[basket].emissions_diff[1]
<ipython-input-9-9096e654f39a>:26: PerformanceWarning: indexing past lexsort depth may impact performance.
  product_3_emissions_diff = df_basket_product.loc[basket].emissions_diff[2]
<ipython-input-9-9096e654f39a>:32: RuntimeWarning: invalid value encountered in double_scalars
  basket_emissions_reduction.append(product_1_emissions_diff/emissions)


In [10]:
basket_substitutions.basket_emissions_reduction.describe()

count    73652.000000
mean         0.196160
std          0.216328
min         -1.373516
25%          0.000000
50%          0.141497
75%          0.309524
max          0.973208
Name: basket_emissions_reduction, dtype: float64

In [11]:
100*basket_substitutions.loc[basket_substitutions.basket_emissions_reduction != 0].basket_emissions_reduction.describe()

count    5.023900e+06
mean     2.875773e+01
std      2.057126e+01
min     -1.373516e+02
25%      1.346675e+01
50%      2.355455e+01
75%      3.949488e+01
max      9.732079e+01
Name: basket_emissions_reduction, dtype: float64

In [12]:
100*(basket_substitutions.basket_emissions_reduction == 0).value_counts() / len(basket_substitutions)

False    68.215634
True     31.784366
Name: basket_emissions_reduction, dtype: float64

In [13]:
(basket_substitutions.basket_emissions_reduction < 0).value_counts() / len(basket_substitutions)

False    0.998385
True     0.001615
Name: basket_emissions_reduction, dtype: float64

In [14]:
(basket_substitutions.basket_emissions_reduction > 0).value_counts() / len(basket_substitutions)

True     0.680405
False    0.319595
Name: basket_emissions_reduction, dtype: float64

# Reduce A Major Category (i.e. Generic Swap Recommedation, meat or dairy)

In [19]:
def substitute_1major_category(category, suffix):
    df_substitute_major = df.groupby(by=['major','r2n_user'], dropna=False).sum()

    df_substitute_major['emissions_diff'] = df_substitute_major['emissions'] - df_substitute_major['new_emissions']

    df_substitute_major = df_household[['previous_emissions']].join(df_substitute_major.loc[category][['emissions_diff']])
    df_substitute_major.loc[df_substitute_major.emissions_diff.isna(), 'emissions_diff'] = 0


    df_substitute_major['new_emissions_' + suffix] = df_substitute_major.previous_emissions - df_substitute_major.emissions_diff
    df_substitute_major['emissions_reduction_' + suffix] = 1 - df_substitute_major['new_emissions_' + suffix]/df_substitute_major['previous_emissions']

    df_substitute_major = df_substitute_major[['new_emissions_' + suffix, 'emissions_reduction_' + suffix]]
    return df_substitute_major


In [20]:
# (i.e. Generic Swap Recommedation, meat)
substitute_meat = substitute_1major_category('Meat and sausages', 'meat')
# (i.e. Generic Swap Recommedation, dair)
substitute_dairy = substitute_1major_category('Milk and milkproducts', 'dairy')